In [1]:
from PIL import Image
import io
import requests
import cv2 

import torchvision.transforms as transforms
import torch
import numpy as np
import matplotlib.pyplot as plt

def pix_per_class(img):
    """returns unique dictionary with class numbers for keys and number of pixels per class as values"""
    unique, counts = np.unique(img, return_counts=True)
    unique = [int(i) for i in unique]
    act_classes = dict(zip(unique, counts))
    
    return act_classes

def most_pixels_nozero(img):
    """Returns class with the most pixels
    Takes in the image, makes a dictionary with keys for class number and value for number of pixels in that class
    Determines which class has the most pixels and returns that class
    Also deletes pixels labeled zero
    """
    unique, counts = np.unique(img, return_counts=True)
    act_classes = dict(zip(unique, counts))
    if 0 in act_classes:
        del act_classes[0]
    v=list(act_classes.values())
    k=list(act_classes.keys())
    return k[v.index(max(v))]
    
def most_pixels(img):
    """Returns class with the most pixels
    Takes in the image, makes a dictionary with keys for class number and value for number of pixels in that class
    Determines which class has the most pixels and returns that class
    """
    unique, counts = np.unique(img, return_counts=True)
    act_classes = dict(zip(unique, counts))
    v=list(act_classes.values())
    k=list(act_classes.keys())
    return k[v.index(max(v))]

def closest(lst, K): 
    """given a list of ints and input int, returns int from list that is closest to input int"""
    return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))] 


"""Section of code does the following:

looping through the contours from the "new" image that only contains 0 and 1 from encompassing 
entire roofs and excluding trees, it:
1. finds the area of each blob, excluding blobs under 100 pixels, because there were a lot of tiny blobs
2. Finds the mean of each blob
3. Append the means of all the blobs to a list creatively named "lst"

Now outside of the loop, it runs the list through a function called "closest", which takes in a list and
single int to represent what the center should be, and returns which item from the list is closest to the center,
The result is a single int saved to "closest_blob"
This is how I find the most centered blob

The 2nd loop just goes through the contours again and checks which contour is equal to the closest blob
then draws that onto the image and saves the contours in right contours

So what goes in is the array from the model inference with the entire image, what comes out is just the 
center roof of that image
"""   

"""creates new image of just 0 and 1 around blobs of roofs, excluding trees and domes
The reason for this is to be able to easily locate the most centered entire roof"""

def center_finder(model_out_array):
    class_ids = list(range(20))
    new = np.zeros(model_out_array.shape[1:], dtype=np.int64)
    new[(model_out_array[0, :, :] > 2) & (model_out_array[0, :, :] < 19)] = 1
    contour, hier = cv2.findContours(
        new, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    external = np.zeros(new.shape)
    lst = []
    middle = np.array([128, 128])

    contour_indices = []

    for i in range(len(contour)):
        if hier[0][i][3] == -1:
            area = cv2.contourArea(contour[i])
            if area > 200:
                coords = contour[i].reshape((-1, 2))
                lst.append(coords.mean(axis=0))
                contour_indices.append(i)
    if len(lst) < 10:
        return 0
    lst = np.array(lst)
    closest_blob = contour_indices[np.argmin(np.sum(np.square(lst - middle), axis=1))]
    right_contours = contour[closest_blob]
    area_middle = cv2.contourArea(contour[closest_blob])
    cv2.drawContours(external, contour, closest_blob, 255, -1)
    seg = np.zeros(model_out_array.shape[1:], dtype=np.int32)
    seg[external == 255] = model_out_array[0][external == 255]
    return seg, right_contours, area_middle

"""
for each contour in the singled out image
1. create new blank image
2. apply contours from that blob onto blank image with pixel values from singled_out image
3. Find most common class of pixels for that class (excluding 0)
4. In another new blank image, called final_product, draw that those contours with the most common pixel
pulled from previous steps

"""

def change_to_most_pix(singled_out, area_middle ):
    sing_contour, sing_hier = cv2.findContours(singled_out,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    #changed6

    new = np.zeros((256, 256) , dtype=np.int64)
    sing_external = np.zeros(new.shape)
    final_product = np.zeros(new.shape)

    for i in range(len(sing_contour)):
        if sing_hier[0][i][3] == -1:
            sing_area = cv2.contourArea(sing_contour[i])
            if sing_area > 120 and sing_area < area_middle:
                sing_external = np.zeros(new.shape)
                single_seg = np.zeros_like((singled_out) , dtype=np.int32)

                cv2.drawContours(sing_external,sing_contour,i,255,-1)
                single_seg[sing_external == 255] = singled_out[sing_external == 255]

                max_pix_this = int(most_pixels_nozero(single_seg))
                cv2.drawContours(final_product,sing_contour,i,max_pix_this,-1)

    return final_product

"""
Removes bad north facing roof directions and turns pixels into kilowatts
in: dict. containing pixels per class
out: dict. containing kwh per class only for best roof faces
Turns pixels to raw DC watts by doing
pixels -> sqft -> panels - > watts DC
only arbitrary for now
"""


def remove_and_reduce(final_dict):
    kw_dict = {}
    #changed7
    arbitrary_num = 350
    max_dc_kw = 0
    # come back and change arbitrary_num once I get what it is
    for i in final_dict:
        if i > 5 and i < 17:
            kw_dict[i] = round(final_dict[i]/arbitrary_num, 2)
            max_dc_kw += kw_dict[i]
    return kw_dict, max_dc_kw

"""Put in secrey key, user address, and dictionary with kw per class
it calls api and gets AC annual kwh for a 10kw system at that address at 180 degrees
then for each actual roof face, takes the total kw and divides 10 (apicall kw)/ actual kilowatts to get divider
then sets divides the ac_annual from the api call by the divider and turns that kw in the dictionary to total 
annual kwh for that roof face, but assuming it is facing south
then it weights the annual kwh for that roof face by a number for the direction

in: secret key, user address, dict. of kw and directions

out: total kwh possible for that roof, and dict. showing kwh per direction
"""
def pvwatts_call(itsasecret, address, kw_dict):
    payload = {'api_key': itsasecret,
    'address':address,
    'system_capacity':10,
    'azimuth':180,
    'tilt':23,
    'array_type':1,
    'module_type':1,
    'losses':7}

    r = requests.get('https://developer.nrel.gov/api/pvwatts/v6.json?', params=payload)
    r_dict = r.json()

    ac_annual = int(r_dict['outputs']['ac_annual'])
    solrad_annual = round(r_dict['outputs']['solrad_annual'],2)

    dummy_kw = 10
    total_annual_kwh = 0
    
    scales = {6: 0.7875, 7: 0.8525,8: 0.91, 9:0.9575, 10:0.99, 11:1, 12:0.99, 13:0.955, 14:0.9075, 15:0.8425, 16:0.775}
    
    for i in kw_dict:
        divider = dummy_kw / kw_dict[i]
        kw_dict[i] = ac_annual / divider
        if i in scales:
            kw_dict[i] *= scales[i]
        total_annual_kwh += kw_dict[i]


    return(int(total_annual_kwh), kw_dict, solrad_annual)



def system_sizer(electric_bill, state, total_annual_kwh, max_dc_kw, solrad_annual):
    state_rates = {'Alaska':21.75,'Alabama':11.85,'Arkansas':9.3,'Arizona':12.21,
                   'California':18.34,'Colorado':11.89,'Connecticut':21.51,
                   'District Of Columbia':13,'Delaware':12.25,'Florida':11.99,
                   'Georgia':10.7,'Hawaii':32.08,'Iowa':11.44,'Idaho':9.83,'Illinois':12.19,
                   'Indiana':11.87,'Kansas':10.29,'Kentucky':10.21,'Louisiana':8.84,
                   'Massachusetts':22.51,'Maryland':13.07,'Maine':17.26,'Michigan':15,
                   'Minnesota':12.79,'Missouri':9.28,'Mississippi':10.95,'Montana':10.83,
                   'North Carolina':10.99,'North Dakota':9.17,'Nebraska':9.6,
                   'New Hampshire':19.96,'New Jersey':15.72,'New Mexico':12.21,'Nevada':11.79,
                   'New York':17.27,'Ohio':11.98,'Oklahoma':8.8,'Oregon':10.63,
                   'Pennsylvania':12.53,'Rhode Island':22.7,'South Carolina':11.89,
                   'South Dakota':10.57,'Tennessee':10.78,'Texas':11.65,'Utah':10.01,
                   'Virginia':11.4,'Vermont':16.73,'Washington':9.31,'Wisconsin':14.01,
                   'West Virginia':9.72,'Wyoming':10.56}
    if state in state_rates:
        this_rate = state_rates[state]
        monthly_needs = int(electric_bill/(this_rate / 100))
        yearly_needs = monthly_needs * 12
        daily_needs = int(yearly_needs / 365)

    system_size_needed = round(daily_needs / solrad_annual * 1.15,1)
    
    if total_annual_kwh > yearly_needs:
        system_size = system_size_needed
        percentage_of_bill = 100
        big_enough = True
    else:
        system_size = max_dc_kw
        percentage_of_bill = int(max_dc_kw / system_size_needed*100)
        big_enough = False
    return daily_needs, this_rate, system_size, percentage_of_bill, big_enough

def transform_image(image_bytes):
    my_transforms = transforms.Compose([transforms.Resize(256),
                                        transforms.ToTensor()])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)


In [2]:
state_rates = {'Alaska':[21.75],
               'Alabama':[11.85],
               'Arkansas':[9.3],
               'Arizona':[12.21],
               'California':[18.34],
               'Colorado':[11.89],
               'Connecticut':[21.51],
               'District Of Columbia':[13],
               'Delaware':[12.25],
               'Florida':[11.99],
               'Georgia':[10.7],
               'Hawaii':[32.08],
               'Iowa':[11.44],
               'Idaho':[9.83],
               'Illinois':[12.19],
               'Indiana':[11.87],
               'Kansas':[10.29],
               'Kentucky':[10.21],
               'Louisiana':[8.84],
               'Massachusetts':[22.51],
               'Maryland':[13.07],
               'Maine':[17.26],
               'Michigan':[15],
               'Minnesota':[12.79],
               'Missouri':[9.28],
               'Mississippi':[10.95],
               'Montana':[10.83],
               'North Carolina':[10.99],
               'North Dakota':[9.17],
               'Nebraska':[9.6],
               'New Hampshire':[19.96],
               'New Jersey':[15.72],
               'New Mexico':12.21,
               'Nevada':11.79,
               'New York':17.27,
               'Ohio':11.98,
               'Oklahoma':8.8,
               'Oregon':10.63,
               'Pennsylvania':12.53,
               'Rhode Island':22.7,
               'South Carolina':11.89,
               'South Dakota':10.57,
               'Tennessee':10.78,
               'Texas':11.65,
               'Utah':10.01,
               'Virginia':11.4,
               'Vermont':16.73,
               'Washington':9.31,
               'Wisconsin':14.01,
               'West Virginia':9.72,
               'Wyoming':10.56}






In [ ]:
from django.shortcuts import render, redirect
from django.contrib import messages
from .models import Signup
from .forms import LeadForm
from main.processing import *
from PIL import Image
import io
import requests
import cv2 
from .unet import ResNetUNet
import torchvision.transforms as transforms
import torch
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
from django.http import HttpResponse
from django.template.loader import get_template
from django.views import View
from xhtml2pdf import pisa
from django.core.mail import send_mail
import base64


def im_2_b64(image):
    buff = BytesIO()
    image.save(buff, format="JPEG")
    img_str = base64.b64encode(buff.getvalue())
    return img_str

def render_to_pdf(template_src, context_dict={}):
        template = get_template(template_src)
        html  = template.render(context_dict)
        result = BytesIO()
        pdf = pisa.pisaDocument(BytesIO(html.encode("ISO-8859-1")), result)
        if not pdf.err:
                return HttpResponse(result.getvalue(), content_type='application/pdf')
        return None

def image_loader(loader, image):
    image = loader(image).float()
    image = torch.tensor(image, requires_grad=True)
    image = image.unsqueeze(0)
    return image
def index(request):
    signup = Signup()
    form = LeadForm()
    if request.method == "POST":
        form = LeadForm(request.POST)
        if form.is_valid():
            device = request.COOKIES['device']
            email = form.cleaned_data.get('email')
            address = form.cleaned_data.get('address')
            electricity_bill = form.cleaned_data.get('electricity_bill')

            obj, created = Signup.objects.update_or_create(
                device=device,
                defaults={'email': email,'address': address, 'electricity_bill': electricity_bill},)

            return redirect(results)
    
    return render(request,
                  template_name="main/index.html",
                  context={'form':form})



def DownloadPDF(request):
    signup = Signup()
    device = request.COOKIES['device']
    customer = Signup.objects.get(device=device)
    encoded = base64.b64encode(customer.img_bytes)
    strencode = str(encoded)
    strencode = strencode[2:-1]
    data = {

        "device":customer.device,
        "customer":customer.address,
        'system_size':customer.system_size,
        'solrad_annual':customer.solrad_annual, 
        'total_annual_kwh':customer.total_annual_kwh, 
        'daily_needs':customer.daily_needs,
        'this_rate': customer.this_rate, 
        'system_size':customer.system_size, 
        'percentage_of_bill': customer.percentage_of_bill,
        'max_dc_kw':customer.max_dc_kw, 
        'cost':customer.cost,
        'image':strencode


        }

    pdf = render_to_pdf('main/pdf_template.html', data)

    response = HttpResponse(pdf, content_type='application/pdf')
    content = f"attachment; filename={customer.system_size}KW_solar_PV.pdf"
    response['Content-Disposition'] = content
    return response

def thanks(request):
    signup = Signup()
    device = request.COOKIES['device']
    customer = Signup.objects.get(device=device)
    address = customer.address
    system_size = customer.system_size
    send_mail(
        "new signup",
        f'{address} needs a {system_size} kw system',
        "floridarenewable@gmail.com",
        ["kyle@gmail.com"])

    return render(request,
                  template_name="main/thanks.html",

                  )


def results(request):

    device = request.COOKIES['device']
    customer = Signup.objects.get(device=device)
    address = customer.address
    electricity_bill  = customer.electricity_bill  
    map_payload = {
        'center': address ,
        'zoom':'20',
        'size':'256x256',
        'format':'jpg',
        'maptype':'satellite',
        'key': dunno

    }

    r = requests.get("https://maps.googleapis.com/maps/api/staticmap?", params=map_payload)
    image_bytes = r.content
    image = Image.open(io.BytesIO(image_bytes))
    data_transforms = transforms.Compose([transforms.Resize(256),transforms.ToTensor(),])
    model = torch.jit.load('/home/kyle/asq_simple/main/optunet93.pt',map_location=torch.device('cpu'))   
    img = image_loader(data_transforms, image)
    out = model(img)
    model_outs = torch.argmax(out,dim=1)
    model_out_array = model_outs.numpy()
    name = 'Kyle'
    state = 'Florida'
    message = client.messages \
                .create(body=f"shape is {model_out_array.shape}",from_='+13367506653',to='+18632548370')
    cen = 0             
    if cen == 0:
        system_size = 0
        solrad_annual = 0
        total_annual_kwh = 0
        daily_needs = 0
        this_rate = 0
        system_size = 0
        percentage_of_bill = 0
        max_dc_kw = 0
        cost = 0
        
    else:
        singled_out, right_contours, area_middle = center_finder(model_out_array)


        final_product = change_to_most_pix(singled_out, area_middle)
        final_dict = pix_per_class(final_product)
        kw_dict, max_dc_kw = remove_and_reduce(final_dict)
        total_annual_kwh, kw_dict, solrad_annual = pvwatts_call(itsasecret, address, kw_dict)
        daily_needs, this_rate, system_size, percentage_of_bill, big_enough = system_sizer(electricity_bill, state,
                                                                            total_annual_kwh, 
                                                                            max_dc_kw,
                                                                            solrad_annual)
 
    pil_image2 = Image.fromarray(np.uint8(model_out_array[0]*12.5)).convert('RGB')
    pil_image = Image.blend(image, pil_image2, 0.8)
    cost = int(system_size * 2500)
    encoded = im_2_b64(pil_image)
    strencode = str(encoded)
    strencode = strencode[2:-1]
    obj, created = Signup.objects.update_or_create(
                    device=device,
                    defaults={'img_bytes':encoded,'system_size':system_size,
                'solrad_annual':solrad_annual, 'total_annual_kwh':total_annual_kwh, 'daily_needs':daily_needs,
                'this_rate': this_rate, 'system_size': system_size, 'percentage_of_bill': percentage_of_bill,
                'max_dc_kw':max_dc_kw, 'cost':cost},)

    return render(request, 
                  template_name="main/results.html",
                  context={'image_bytes':strencode,'customer':customer,'address':address,'system_size':system_size,
                  'solrad_annual':solrad_annual, 'total_annual_kwh':total_annual_kwh, 'daily_needs':daily_needs,
                  'this_rate': this_rate, 'system_size': system_size, 'percentage_of_bill': percentage_of_bill,
                  'max_dc_kw':max_dc_kw, 'cost':cost })


